In [1]:
import sys
sys.path.append('/home/vinnylg/projects/covid19datascience')
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels
from bulletin.utils.static import Regionais

In [4]:
read_cc = True and False
read_tb = True and False

In [5]:
cc = CasosConfirmados()
if read_cc:
    cc.read('/home/vinnylg/projects/covid19datascience/input/Casos confirmados PR.xlsx')
    cc.save()
else:
    cc.load()

loading Casos Confirmados time elepsed:00:00:01


In [6]:
casosc = cc.get_casos()
del casosc['identificacao']
print(f"casosc len: {len(casosc)}\n")
print(casosc.columns.values)
casosc.loc[casosc['nome']=='ONILSON CANDIDO DE JESUS','rs'] = 10 #TODO

casosc len: 982534

['ordem' 'ibge_res_pr' 'rs_res_pr' 'nome' 'sexo' 'idade' 'mun_resid'
 'mun_atend' 'rs' 'laboratorio' 'dt_diag' 'comunicacao' 'is' 'obito'
 'data_obito' 'excluir' 'hash' 'hash_less' 'hash_more' 'hash_atend'
 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


In [7]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
print(casost.columns.values)

casost len: 982534

['identificacao' 'ibge_res_pr' 'ibge_atend_pr' 'nome' 'sexo' 'idade'
 'mun_resid' 'mun_atend' 'laboratorio' 'dt_diag' 'dt_notificacao'
 'dt_inicio_sintomas' '_dia_isolamento' 'dt_viagem_retorno' 'local_viagem'
 'exposicao' 'data_de_internamento' 'local' 'clinico_uti' 'dt_alta'
 'obito' 'dt_obito' 'muni_res_pr' 'hist_viagem' 'casos_novos' 'telefone'
 'dt_atualizacao' 'fonte' 'status' 'eas' 'privado_sus' 'uti'
 'dt_internamento' 'evolucao' 'dt_evolucao' 'temp_permanencia'
 'cod_laboratorio' 'idade_original' 'dt_com_obito' 'dt_com_recuperado'
 'idade_mais' 'idade_menos' 'hash' 'hash_less' 'hash_more' 'hash_atend'
 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


In [8]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [9]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

23773
958761


In [10]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

981513
1638


In [11]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

983151
1


In [12]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

983151
1


In [13]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

983152
0


,ordem,ibge_res_pr,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [14]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [15]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag']==ordem_duplicado['hash_diag'])

741 741


True

In [16]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]
casosct.loc[casosct['comunicacao']==pd.to_datetime('2020-03-11'),'comunicacao'] = pd.to_datetime('2020-03-12')

In [17]:
print(len(casosct))
casosct = casosct.drop_duplicates('identificacao',keep='last')
print('casos:',len(casosct))
print('obitos:',len(casosct.loc[casosct['obito']=='SIM']))

983152
casos: 982534
obitos: 23777


In [18]:
try:
    casosct = casosct_bkp
except:
    casosct_bkp = casosct

In [19]:
casosct = casosct.sort_values(['comunicacao','ibge_res_pr','nome','idade']).copy()
casosct['uf_resid'] = 'PR'
casosct.loc[casosct['rs']==99,'uf_resid'] = casosct.loc[casosct['rs']==99,'mun_resid'].apply(lambda x: x.split('/')[-1])

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

In [20]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

casosct['uf_pr'] = 'FORA'
casosct.loc[casosct['rs']!=99,'uf_pr'] = 'PR'

total = casosct.groupby(['evolucao','uf_pr'])[['identificacao']].count().unstack().droplevel(0,1)[['PR','FORA']]
total['TOTAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name='TOTAL'))
total.astype(float)

uf_pr,PR,FORA,TOTAL
evolucao,,,
1,696.293,2.142,698.435
2,23.631,146,23.777
3,256.861,3.461,260.322
TOTAL,976.785,5.749,982.534


In [21]:
casosct['municipio_pr'] = 'FORA'
casosct.loc[casosct['rs']!=99,'municipio_pr'] = casosct.loc[casosct['rs']!=99,'mun_resid']

regionais = Regionais()
regionais = regionais.append(pd.Series([999999,99,'FORA','OUTROS','9999999'], index=regionais.columns), ignore_index=True)
macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(regionais['nm_macro'].unique())])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])

regionais['ibge'] = regionais['ibge'].apply(int)
regionais['nm_macro'] = regionais['nm_macro'].apply(lambda x: macros_labels_idx[x])

casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)

casosct = pd.merge(casosct,regionais[['ibge','nm_reg','nm_macro']].rename(columns={'ibge':'ibge_res_pr','nm_reg':'reg','nm_macro':'macro'}),how='left',on='ibge_res_pr')

total_municipios = casosct.groupby(['macro','rs','reg','ibge_res_pr','municipio_pr','evolucao'])[['identificacao']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios = total_municipios.append(pd.Series(total_municipios.sum(0),name=('','TOTAL','GERAL','','')))

pr = total_municipios.loc[('','TOTAL','GERAL','','')].values[0] - total_municipios.loc[4].values[0]

total_municipios = total_municipios.append(pd.Series(pr, index=total_municipios.columns, name=(('','TOTAL','PR','',''))))

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)
total_municipios

evolucao                                           CASOS  OBITOS  RECUPERADOS  \
macro  rs    reg        ibge_res_pr municipio_pr                                
LESTE  1      PARANAGUA 410120      ANTONINA       2.135      55        1.942   
                        410950      GUARAQUECABA     547      14          466   
                        410960      GUARATUBA      4.041     137        3.692   
                        411570      MATINHOS       2.391      77        2.088   
                        411620      MORRETES       1.690      40        1.316   
...                                                  ...     ...          ...   
OESTE  20    TOLEDO     412770      TOLEDO        18.817     285       17.658   
                        412795      TUPASSI          722      23          625   
OUTROS 99    FORA       999999      FORA           5.749     146        2.142   
       TOTAL GERAL                               982.534  23.777      698.435   
             PR                                  976.785 982.388      980.392   

evolucao                                          ATIVOS  
macro  rs    reg        ibge_res_pr municipio_pr          
LESTE  1      PARANAGUA 410120      ANTONINA         138  
                        410950      GUARAQUECABA      67  
                        410960      GUARATUBA        212  
                        411570      MATINHOS         226  
                        411620      MORRETES         334  
...                                                  ...  
OESTE  20    TOLEDO     412770      TOLEDO           874  
                        412795      TUPASSI           74  
OUTROS 99    FORA       999999      FORA           3.461  
       TOTAL GERAL                               260.322  
             PR                                  979.073  

[404 rows x 4 columns]

In [22]:
total_regionais = casosct.groupby(['rs','reg','evolucao'])[['identificacao']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0).astype(float)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais = total_regionais.append(pd.Series(total_regionais.sum(0),name=('TOTAL','GERAL')))

pr = total_regionais.loc[('TOTAL','GERAL')].values[0] - total_regionais.loc[99].values[0]

total_regionais = total_regionais.append(pd.Series(pr, index=total_regionais.columns, name=(('TOTAL','PR'))))
total_regionais

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,reg,,,,
1,PARANAGUA,31.435,754,16.909,13.772
2,METROPOLITANA,298.033,8.678,215.494,73.861
3,PONTA GROSSA,56.446,1.324,38.533,16.589
4,IRATI,11.264,196,9.381,1.687
5,GUARAPUAVA,28.665,569,17.634,10.462
6,UNIAO DA VITORIA,8.907,176,7.003,1.728
7,PATO BRANCO,25.711,574,21.905,3.232
8,FRANCISCO BELTRAO,34.933,601,28.210,6.122
9,FOZ DO IGUACU,50.967,1.038,41.000,8.929


In [24]:
casos_hashes = casosct[['identificacao', 'hash', 'hash_less', 'hash_more', 'hash_atend', 'hash_diag']]
casos_hashes.to_excel('casos_hashes.xlsx', index=False)

obitosct = casosct.loc[casosct['evolucao']==2]
obitos_hashes = obitosct[['identificacao', 'hash', 'hash_less', 'hash_more', 'hash_atend', 'hash_diag', 'hash_obito']]
obitos_hashes.to_excel('obitos_hashes.xlsx', index=False)
    
casosct = casosct[['identificacao','uf_resid','ibge_res_pr','ibge_atend_pr', 'rs', 'nome', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'laboratorio', 'dt_diag', 'comunicacao', 'is','obito', 'data_obito', 'dt_com_obito', 'status', 'dt_com_recuperado', 'evolucao', 'data_evolucao', 'data_com_evolucao']]
obitosct = obitosct[['identificacao', 'uf_resid', 'ibge_res_pr', 'rs', 'nome', 'sexo', 'idade', 'mun_resid', 'dt_diag','data_evolucao', 'data_com_evolucao', 'excluir']]

writer = pd.ExcelWriter(join("output","novos_casos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')
casosct.to_excel(writer,'casos', index=False)
obitosct.to_excel(writer,'obitos',index=False)

writer.save()